In [103]:
import pandas as pd
import numpy as np
import pymysql
from datetime import datetime, timedelta
import warnings
import requests
import json
import fire
warnings.filterwarnings('ignore')

## Datos a usar

In [116]:
def setcut_prob_quash(p, grupo_ = True):
    p = p*100
    cuts = np.array(sorted([100, 96.8, 93.6, 84.1, 75.4, 50.2, 0.00]))
    cosechas = np.array(sorted([1, 1.98, 4.06, 6.11, 7.65, 11.8, 14.76], reverse = True))
                                 
    
    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))
    
    for cut in cuts:
        if cut > p:
            break
            
    grupo = np.where(cuts == cut)[0][0] 
    if grupo_:
        return grupo
    else:
        return cosechas[grupo - 1]
def excel_to_datetime(excel_date):
    # Excel date base is January 1, 1900
    base_date = pd.Timestamp('1900-01-01')
    # Convert the Excel date number to a pandas Timedelta
    delta = pd.Timedelta(days=excel_date - 2)  # Subtract 2 days: one for the Excel leap year bug and one for inclusive counting
    # Add the delta to the base date
    result_date = base_date + delta
    return result_date

In [113]:
data_perdida_cosecha = pd.read_excel("../matriz_modelo_riesgo/data/240531 PERDIDA DE COSECHA.xlsb", sheet_name = 'COLOCACION', header = 1)
data_perdida_cosecha.columns = [k.lower().replace(" ", "_") for k in data_perdida_cosecha.columns]
data_perdida_cosecha.rename(columns={"id_dist.":"id_distribuidor"}, inplace = True)

In [114]:
def primer_no_nulo_fila(fila):
    return fila[fila.first_valid_index()]

# Aplicar la función a cada fila del DataFrame
data_perdida_cosecha['id_distribuidor_usar'] = data_perdida_cosecha[['id_distribuidor_linea_de_credito', 'id_distribuidor']].apply(primer_no_nulo_fila, axis=1)

In [117]:
data_perdida_cosecha_usar = data_perdida_cosecha[['id_distribuidor_usar', 'id_distribuidor','perdida_de_cosecha', 'capital', 'tasa', 'inicio_credito', 'fin_credito']]
data_perdida_cosecha_usar.inicio_credito = data_perdida_cosecha_usar.inicio_credito.apply(excel_to_datetime)
data_perdida_cosecha_usar.fin_credito = data_perdida_cosecha_usar.fin_credito.apply(excel_to_datetime)
data_perdida_cosecha_usar.head()

data_perdida_cosecha_usar["perdido_capital"] = abs(data_perdida_cosecha_usar.perdida_de_cosecha * data_perdida_cosecha_usar.capital)
data_perdida_cosecha_usar["tasa_ponderada"] = abs(data_perdida_cosecha_usar.tasa * data_perdida_cosecha_usar.capital)

data_perdida_cosecha_solo = (data_perdida_cosecha_usar.groupby("id_distribuidor_usar").perdido_capital.sum() / data_perdida_cosecha_usar.groupby("id_distribuidor_usar").capital.sum()).reset_index()
data_perdida_cosecha_solo.columns =['id_distribuidor_usar', 'perdida_ponderada']
data_perdida_tasa_solo = (data_perdida_cosecha_usar.groupby("id_distribuidor_usar").tasa_ponderada.sum() / data_perdida_cosecha_usar.groupby("id_distribuidor_usar").capital.sum()).reset_index()
data_perdida_tasa_solo.columns = ['id_distribuidor_usar', 'tasa_ponderada']

data_perdida_cosecha_tasa_usar_2 = data_perdida_tasa_solo.merge(data_perdida_cosecha_solo)
data_perdida_cosecha_tasa_usar_2.head()

ids_distribuidores = data_perdida_cosecha_usar[['id_distribuidor_usar', 'id_distribuidor']].drop_duplicates("id_distribuidor_usar")

data_perdida_cosecha_tasa_usar = data_perdida_cosecha_tasa_usar_2.merge(ids_distribuidores, 
                                       on='id_distribuidor_usar')

In [122]:
data_perdida_cosecha_tasa_usar.id_distribuidor_usar = data_perdida_cosecha_tasa_usar.id_distribuidor_usar.astype(int)

In [129]:
ids = "("+", ".join(list(data_perdida_cosecha_tasa_usar.id_distribuidor_usar.astype(str)))+")"

## Code

In [94]:
URL = "https://api.quash.ai"
DATA = "./data/datos.json"
ENDPOINT = "/scoring"
APIKEY = "Ku9sm6Nr6XAEXlWfLNyPmDhkTvTa4EnS"

In [135]:
sql_demo = f"""
select * from (select 
    id as 'APPLICANT_ID', 
    monto_mensual as ingreso_mensual,
    gastos_familiares,
    otros_ingresos,
    replace(replace(replace(ingreso_neto,"$",""), ",",""), ".00","") as ingreso_neto,
    dependientes,
    ventas,
    FLOOR(DATEDIFF(CURDATE(), fecha_nacimiento) / 365.25) AS edad,
    monto_mensual,
    compras,
    delegacion,
    delegacion_negocio,
    escolaridad,
    estado,
    estado_civil,
    estado_negocio,
    sexo,
    sucursal,
    tipo_negocio,
    tipo_vivienda
from cc_prospectos_demograficos

UNION

select 
    id as 'APPLICANT_ID', 
    ingreso_mensual,
    gastos_familiares,
    otros_ingresos,
    replace(replace(replace(ingreso_neto,"$",""), ",",""), ".00","") as ingreso_neto,
    dependientes,
    ventas,
    FLOOR(DATEDIFF(CURDATE(), fecha_nacimiento) / 365.25) AS edad,
    monto_mensual_venta,
    compras,
    delegacion,
    delegacion_negocio,
    escolaridad,
    estado,
    estado_civil,
    estado_negocio,
    sexo,
    sucursal,
    tipo_negocio,
    tipo_vivienda
from demograficos_distribuidores) as pen
where pen.APPLICANT_ID in {ids}
"""

In [136]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                               charset='utf8',db = 'volana')
cursor = cnx.cursor()
cursor.execute(sql_demo)
data_demo = cursor.fetchall()
data_demo = pd.DataFrame(data_demo, columns=[str(i[0]).upper() for i in cursor.description])
cnx.close()

## Numéricas

In [138]:
for col in ['INGRESO_MENSUAL', 'GASTOS_FAMILIARES',
       'OTROS_INGRESOS', 'INGRESO_NETO', 'DEPENDIENTES', 'VENTAS', 'EDAD',
       'MONTO_MENSUAL', 'COMPRAS']:
    
    data_demo[col] = data_demo[col].fillna(0)
    data_demo[col] = data_demo[col].apply(lambda x: float(str(x).replace("$","").replace(" ANOS","").replace("|","").replace(",","")))

    data_demo[col] = data_demo[col].fillna(0)
    data_demo[col] = data_demo[col].astype(float)
    data_demo[col] = data_demo[col].fillna(0).astype(int)

## Categóricas

In [139]:
DELGACIONES = [
    "CENTRO", "CHALCO", "CHICOLOAPAN", "CHIMALHUACAN", "IXTAPALUCA", "IZTAPALAPA", "LA_PAZ", "NEZAHUALCOYOTL",
    "VALLE_DE_CHALCO_SOLIDARIDAD", "ACAMBAY", "ACOLMAN", "AMATLAN_DE_LOS_REYES", "AMECAMECA", "ATENCO", "ATLAUTLA",
    "ATOYAC", "ATZACAN", "AYAPANGO", "BENITO_JUAREZ", "CAMERINO_Z._MENDOZA", "CARDENAS", "CHIAUTLA", "CHICONCUAC",
    "CHOCAMAN", "COATZACOALCOS", "COMALCALCO", "CORDOBA", "COSCOMATEPEC", "COSOLEACAQUE", "CUAUHTEMOC", "CUITLAHUAC",
    "CUNDUACAN", "ECATEPEC_DE_MORELOS", "ECATZINGO", "FORTIN", "GUADALAJARA", "GUSTAVO_A._MADERO", "HUATUSCO",
    "HUILOAPAN_DE_CUAUHTEMOC", "HUIMANGUILLO", "HUIXQUILUCAN", "IXHUATLANCILLO", "IXTACZOQUITLAN",
    "IXTLAHUACAN_DE_LOS_MEMBRILLOS", "IZTACALCO", "JUCHITEPEC", "LA_PERLA", "MACUSPANA", "MARIANO_ESCOBEDO",
    "MILPA_ALTA", "MINATITLAN", "NACAJUCA", "NANCHITAL_DE_LAZARO_CARDENAS_DEL_RIO", "NARANJAL", "NOGALES",
    "OMEALCA", "ORIZABA", "OZUMBA", "PAPALOTLA", "RAFAEL_DELGADO", "RIO_BLANCO", "TEMAMATLA", "TENANGO_DEL_AIRE",
    "TEOTIHUACAN", "TEPETLAOXTOC", "TEPETLIXPA", "TEPOTZOTLAN", "TEQUILA", "TEXCOCO", "TEZOYUCA", "TLAHUAC",
    "TLAJOMULCO_DE_ZUNIGA", "TLALMANALCO", "TLAQUEPAQUE", "TLILAPAN", "TOMATLAN", "TONALA", "VENUSTIANO_CARRANZA",
    "XOCHIMILCO", "ZAPOPAN", "ZAPOTLANEJO"
]
ESCOLARIDADES = [
    "BACHILLERATO", "LICENCIATURA", "PRIMARIA", "SECUNDARIA", "TECNICO", "OTRO", "SIN_ESTUDIOS"
]
ESTADOS = [
    "DISTRITO_FEDERAL", "JALISCO", "MEXICO", "TABASCO", "VERACRUZ_DE_IGNACIO_DE_LA_LLAVE", "QUINTANA_ROO"
]
ESTADOS_CIVILES = [
    "CASADO", "SOLTERO", "UNION LIBRE", "VIUDO", "DIVORCIADO", "SEPARADO"
]
SUCURSALES = [
    "AMECAMECA", "CARDENAS", "CHALCO", "CHIMALHUACAN_2", "COATZACOALCOS", "FORTIN", "IXTAPALUCA", "IXTAPALUCA_2",
    "NEZAHUALCOYOTL_2", "ORIZABA", "TEXCOCO", "TONALA", "TONALA_2", "VILLAHERMOSA", "IZTAPALAPA", "MILPA_ALTA",
    "TEXCOCO_2", "TLAHUAC"
]
TIPOS_NEGOCIO = [
    "ABARROTES/MISCELANEAS/PERECEDEROS", "BELLEZA/SALON/ESTETICA/BARBERIA", "CALZADO_REPARACION_Y_VENTA",
    "FABRICA/TALLER/ELABORACION/REPARACION", "FRUTAS/VERDURAS/SEMILLAS/CAMPO/AGROPECUARIAS",
    "JOYERIA/BISUTERIA/FUNDICION", "VENTA_DE_ALIMENTOS", "VENTA_DE_ROPA", "VENTA_POR_CATALOGO", "AGUA/POTABLE/RESIDUAL",
    "AMEDICINA/SALUD/FARMACIA", "ANIMALES/VETERINARIA", "ARTE/ARTESANIAS", "CAFEINTERNET",
    "CARNICERIA/POLLERIA/RASTRO", "CERRAJERIA", "COMBUSTIBLES/GAS/GASOLINA", "DEPORTES/GIMNASIO",
    "DESPERDICIOS/RESIDUOS/RECICLAJE", "DISTRIBUIDOR_CON_EXPERIENCIA", "EDUCACION/ESCUELAS/CENTRO_DE_CAPACITACION",
    "ELECTRICISTA", "EVENTOS/SALONES/SILLAS/MESAS", "FLORES/ARREGLOS", "FOTOGRAFIA/VIDEO", "HERRERIA",
    "IMPRENTA/SERIGRAFIA/ROTULOS", "JARCIERIA", "LAVANDERIA/TINTORERIA/PLANCHADURIA", "MADERA/CARPINTERIA/EBANISTA",
    "MECANICA_AUTOMOTRIZ/INDUSTRIAL", "PANADERIA", "PAPELERIA", "PLOMERIA", "TLAPALERIA/FERRETERIA/MATERIALES_DE_CONSTRUCCION",
    "TORTILLERIA/MOLINO", "VENTA_DE_AUTOS/MOTOCICLETAS/LOTES/AUTOPARTES", "VENTA_DE_CALZADO",
    "VENTA_DE_CELULARES/TECNOLOGIA", "VENTA_DE_COMIDA", "VENTA_DE_MUEBLES/ELECTRONICOS/ELECTRODOMESTICOS"
]
TIPOS_VIVIENDA = [
    "FAMILIAR", "PROPIA", "RENTADA", "OTRO"
]

def levenshtein_distance(word1, word2):
    len1, len2 = len(word1), len(word2)
    
    # Crear una matriz para almacenar las distancias parciales
    dp = [[0] * (len2 + 1) for _ in range(len1 + 1)]

    # Inicializar la primera fila y columna
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j

    # Calcular las distancias parciales
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = 0 if word1[i - 1] == word2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,      # Eliminación
                           dp[i][j - 1] + 1,      # Inserción
                           dp[i - 1][j - 1] + cost)  # Sustitución

    # El resultado final está en la esquina inferior derecha de la matriz
    return dp[len1][len2]


def MostSimilar(word, list_):
    word = str(word).upper()
    min_distance = np.inf
    index = 0
    for i, element in enumerate(list_):
        d = levenshtein_distance(element, word)
        if d < min_distance:
            min_distance = d
            index = i
            
    return list_[index]

In [140]:
data_demo.DELEGACION = data_demo.DELEGACION.apply(MostSimilar, list_ = DELGACIONES)
data_demo.DELEGACION_NEGOCIO = data_demo.DELEGACION_NEGOCIO.apply(MostSimilar, list_ = DELGACIONES)
data_demo.ESCOLARIDAD = data_demo.ESCOLARIDAD.apply(MostSimilar, list_ = ESCOLARIDADES)
data_demo.ESTADO = data_demo.ESTADO.apply(MostSimilar, list_ = ESTADOS)
data_demo.ESTADO_NEGOCIO = data_demo.ESTADO_NEGOCIO.apply(MostSimilar, list_ = ESTADOS)
data_demo.SUCURSAL = data_demo.SUCURSAL.apply(MostSimilar, list_ = SUCURSALES)
data_demo.TIPO_NEGOCIO = data_demo.TIPO_NEGOCIO.apply(MostSimilar, list_ = TIPOS_NEGOCIO)
data_demo.TIPO_VIVIENDA = data_demo.TIPO_VIVIENDA.apply(MostSimilar, list_ = TIPOS_VIVIENDA)
data_demo.ESTADO_CIVIL = data_demo.ESTADO_CIVIL.apply(MostSimilar, list_ = ESTADOS_CIVILES)

## Guardando el json con el response

In [148]:
consulta = data_demo.to_dict('records')

In [150]:
prefix = '{"data": '
suffix = ', "customer": "volana"}'

# Unimos el prefijo, el diccionario serializado como JSON y el sufijo
for i in range(500):
    json_data = prefix + json.dumps(consulta[100*i:100*(i+1)]) + suffix

    file_path = f"./evaluacion_clientes_2023/data/json/datos_finales_hora_si_{i}.json"

    with open(file_path, "w") as json_file:
        json_file.write(json_data)

In [151]:
resultado_final_hora_si = []

In [293]:
def predict_test(url=URL,data=DATA,endpoint=ENDPOINT):
    with open(data,'r') as _:
        
            global resultado_final_hora_si
            j_data = json.load(_)
            headers = {'content-type': 'application/json',

            'Accept-Charset': 'UTF-8',
            'apikey': APIKEY,
            'x-customer': 'volana'}


            r = requests.post(url + endpoint, data=json.dumps(j_data), headers=headers)
            resultado_final_hora_si.append(r.text)
            print(r, r.text)
        

i = 0
for i in range(81, 82):
    DATA = f"./evaluacion_clientes_2023/data/json/datos_finales_hora_si_{i}.json"
    print(DATA)
    try:
        fire.Fire(predict_test)
    except Exception as e:
        pass
    
    print(f"Next {i}")
    
    time.sleep(1)

./evaluacion_clientes_2023/data/json/datos_finales_hora_si_81.json
<Response [500]> Internal Server Error


ERROR: Could not consume arg: -f
Usage: ipykernel_launcher.py -

For detailed information on this command, run:
  ipykernel_launcher.py - --help


FireExit: 2

In [294]:
len(resultado_final_hora_si)
#resultado_final_hora_si

111

In [291]:
res = pd.DataFrame()
for i in range(len(resultado_final_hora_si)):
    try:
        res = pd.concat([res, pd.DataFrame(eval(resultado_final_hora_si[i])['prediction'])])
    except:
        pass
    
res.to_csv("./evaluacion_clientes_2023/data/resultado_modelo_quash_api.csv", index=False)

In [299]:
res

,label,approval_score,applicant_id
0,RECHAZADO,0.578301,145442
1,APROBADO,0.991986,145444
2,APROBADO,0.994773,157169
3,RECHAZADO,0.564043,157171
4,RECHAZADO,0.837451,157178
...,...,...,...
85,RECHAZADO,0.059387,156330
86,RECHAZADO,0.219662,156834
87,RECHAZADO,0.425555,156931
88,RECHAZADO,0.167120,157057


In [304]:
res.applicant_id = res.applicant_id.astype(str)

data_perdida_cosecha_tasa_usar.id_distribuidor_usar = data_perdida_cosecha_tasa_usar.id_distribuidor_usar.astype(str)

In [306]:
data_general = data_perdida_cosecha_tasa_usar.merge(res, left_on = 'id_distribuidor_usar', right_on = 'applicant_id')

In [308]:
data_general["perdida_cosecha_esperada_quash"] = data_general.approval_score.apply(setcut_prob_quash, grupo_ = False)
data_general["grupo_riesgo"] = data_general.approval_score.apply(setcut_prob_quash, grupo_ = True)

In [310]:
data_general.to_csv("./evaluacion_clientes_2023/data/data_riesgo_quash_2023.csv", index = False)